In [3]:
# install package
!apt-get update
!apt-get upgrade
!apt-get install build-essential
!apt-get install cmake git libgtk2.0-dev pkg-config libavcodec-dev libavformat-dev libswscale-dev

# clone darknet repo
!git clone https://github.com/AlexeyAB/darknet


Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:4 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:8 http://security.ubuntu.com/ubuntu focal-security/universe amd64 Packages [1,057 kB]
Get:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease [18.1 kB]
Get:10 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Get:12 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [2,774 kB]
Get:13 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 Pack

In [4]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile
!sed -i 's/LIBSO=0/LIBSO=1/' Makefile


/content/darknet


In [5]:
# make darknet
!make


mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -DCUDNN_HALF -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -rdynamic -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -DCUDNN_HALF -fPIC -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
  946 |                 float rgb[3];
      |                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1147:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
 1147 |  

In [18]:
# Mount Google Drive (optional, if your dataset is stored in Google Drive)
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [19]:
# Copy dataset and annotations to the darknet directory
!cp /content/drive/MyDrive/PPE-2.v1i.darknet.zip /content/darknet
!cp /content/drive/MyDrive/PPE-2.v1i.darknet.zip /content/darknet

In [20]:
# Unzip the dataset and annotations
!unzip PPE-2.v1i.darknet.zip
# !unzip your_annotations_path.zip

Archive:  PPE-2.v1i.darknet.zip
 extracting: README.dataset.txt      
 extracting: README.roboflow.txt     
   creating: test/
 extracting: test/20220721_161942_jpg.rf.5686b9a23e8aa5cc694829d3967998e6.jpg  
 extracting: test/20220721_161942_jpg.rf.5686b9a23e8aa5cc694829d3967998e6.txt  
 extracting: test/20220721_162604_jpg.rf.8dcc00e484617534b6ab1352ede398d2.jpg  
 extracting: test/20220721_162604_jpg.rf.8dcc00e484617534b6ab1352ede398d2.txt  
 extracting: test/IMG-20221020-WA0121_jpg.rf.23d9fecc5e47304d5a056155271545a8.jpg  
 extracting: test/IMG-20221020-WA0121_jpg.rf.23d9fecc5e47304d5a056155271545a8.txt  
 extracting: test/KakaoTalk_20221018_001605157_12_jpg.rf.9224fe92e43b01213844a74b6358c67f.jpg  
 extracting: test/KakaoTalk_20221018_001605157_12_jpg.rf.9224fe92e43b01213844a74b6358c67f.txt  
 extracting: test/KakaoTalk_20221018_001605157_12_jpg.rf.db43b0a8fe7029cf40585a100abedf2f.jpg  
 extracting: test/KakaoTalk_20221018_001605157_12_jpg.rf.db43b0a8fe7029cf40585a100abedf2f.txt  
 

In [12]:
# Create train.txt file (list of training images)
import os

with open('train.txt', 'w') as f:
  for filename in os.listdir('/content/darknet/train'):
    if filename.endswith('.jpg'):
      f.write('/content/darknet/train' + filename + '\n')



In [14]:
# Create obj.names file (list of class names)
class_names = ['helm safety', 'masker', 'baju_pelindung','sepatu safety']

In [15]:
with open('obj.names', 'w') as f:
  for class_name in class_names:
    f.write(class_name + '\n')

In [16]:
# Create obj.data file (YOLO configuration file)
with open('obj.data', 'w') as f:
  f.write('classes = ' + str(len(class_names)) + '\n')
  f.write('train = train.txt\n')
  f.write('names = obj.names\n')
  f.write('backup = backup/')

In [22]:
# Create obj.zip file
!zip obj.zip /content/darknet/train -r

updating: content/darknet/train/ (stored 0%)
  adding: content/darknet/train/image_254_jpg.rf.260f99ab25722c0f24b40871a5a914e2.txt (deflated 58%)
  adding: content/darknet/train/20220621_132920_jpg.rf.8d9cc246bceecc2e02fbf4255189ab1b.jpg (deflated 1%)
  adding: content/darknet/train/image_209_jpg.rf.2ab4250e99afb93216f2453f5aebc3a4.jpg (deflated 0%)
  adding: content/darknet/train/images-50-_jpg.rf.4201b05f2b00a985dded3cf29e76d156.txt (deflated 51%)
  adding: content/darknet/train/KakaoTalk_20221018_001312059_02_jpg.rf.311c305ab98c911d59ed5433ab709afa.txt (deflated 61%)
  adding: content/darknet/train/IMG-20221020-WA0129_jpg.rf.83d4cf0d1af6b026b864c1c2570b2fd4.txt (deflated 28%)
  adding: content/darknet/train/image_256_jpg.rf.ed31f3bb2305aacd9886b8d9a8c810ac.jpg (deflated 0%)
  adding: content/darknet/train/image_184_jpg.rf.8dcfbc8ee99ddc5ce59f0264ae37d9ea.txt (deflated 58%)
  adding: content/darknet/train/images-2022-07-04T013058_jpg.rf.11c46b8fef9aeb3efb7146e8fb47d990.jpg (deflated 

In [21]:
# get yolov3 pretrained coco dataset weights
!wget https://pjreddie.com/media/files/yolov3-tiny.weights


--2023-06-06 18:17:28--  https://pjreddie.com/media/files/yolov3-tiny.weights
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 35434956 (34M) [application/octet-stream]
Saving to: ‘yolov3-tiny.weights.1’

yolov3-tiny.weights 100%[===================>]  33.79M  12.2MB/s    in 2.8s    

2023-06-06 18:17:32 (12.2 MB/s) - ‘yolov3-tiny.weights.1’ saved [35434956/35434956]



In [23]:
# Convert annotations to Darknet format
!python voc_label.py

python3: can't open file '/content/darknet/voc_label.py': [Errno 2] No such file or directory


In [26]:
# Start training
!./darknet detector train obj.data yolov3-tiny.cfg yolov3-tiny.weights -dont_show

 CUDA-version: 11080 (12000), cuDNN: 8.7.0, CUDNN_HALF=1, GPU count: 1  
 CUDNN_HALF=1 
 OpenCV version: 4.2.0
valid: Using default 'train.txt'
yolov3-tiny
 0 : compute_capability = 750, cudnn_half = 1, GPU: Tesla T4 
net.optimized_memory = 0 
mini_batch = 1, batch = 64, time_steps = 1, train = 1 
   layer   filters  size/strd(dil)      input                output
   0 Create CUDA-stream - 0 
 Create cudnn-handle 0 
conv     16       3 x 3/ 1    416 x 416 x   3 ->  416 x 416 x  16 0.150 BF
   1 max                2x 2/ 2    416 x 416 x  16 ->  208 x 208 x  16 0.003 BF
   2 conv     32       3 x 3/ 1    208 x 208 x  16 ->  208 x 208 x  32 0.399 BF
   3 max                2x 2/ 2    208 x 208 x  32 ->  104 x 104 x  32 0.001 BF
   4 conv     64       3 x 3/ 1    104 x 104 x  32 ->  104 x 104 x  64 0.399 BF
   5 max                2x 2/ 2    104 x 104 x  64 ->   52 x  52 x  64 0.001 BF
   6 conv    128       3 x 3/ 1     52 x  52 x  64 ->   52 x  52 x 128 0.399 BF
   7 max                2

In [27]:
from google.colab import files
files.download("/content/darknet/backup/yolov3-tiny_final.weights")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>